# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [3]:
FEATURES_TRAIN_PATH="MNIST/digit_xtrain.csv"
FEATURES_TEST_PATH="MNIST/digit_xtest.csv"
TARGET_TRAIN_PATH="MNIST/digit_ytrain.csv"
TARGET_TEST_PATH="MNIST/digit_ytest.csv"

LOGGING_PATH='tensorboard_mnist_digit_log/'

NR_CLASSES=10
VALIDATION_SIZE=10000
IMAGE_WIDTH=28
IMAGE_HEIGHT=28
CHANNELS=1
TOTAL_INPUT=IMAGE_HEIGHT*IMAGE_WIDTH*CHANNELS
print(TOTAL_INPUT)

784


# Get the Data and Print Shape

In [4]:
%%time

target_train_all= np.loadtxt(TARGET_TRAIN_PATH, delimiter=',', dtype=int)
print(target_train_all.shape)
target_test= np.loadtxt(TARGET_TEST_PATH, delimiter=',', dtype=int)
print(target_test.shape)

(60000,)
(10000,)
CPU times: user 567 ms, sys: 12.8 ms, total: 580 ms
Wall time: 592 ms


In [5]:
%%time
features_train_all= np.loadtxt(FEATURES_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 1min 13s, sys: 5.33 s, total: 1min 18s
Wall time: 1min 22s


In [6]:
%%time
features_test= np.loadtxt(FEATURES_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 11.4 s, sys: 364 ms, total: 11.8 s
Wall time: 11.9 s


# Explorar

In [7]:
t=[1,2,3,4,5,6,7,8,9]
print('First three: ',t[:3])
print('Other then First three: ',t[3:])

First three:  [1, 2, 3]
Other then First three:  [4, 5, 6, 7, 8, 9]


In [8]:
print("features_train_all shape : ",features_train_all.shape)
print("target_train_all shape : ",target_train_all.shape)
print("features_test shape : ",features_test.shape)
print("target_test shape : ",target_test.shape)

features_train_all shape :  (60000, 784)
target_train_all shape :  (60000,)
features_test shape :  (10000, 784)
target_test shape :  (10000,)


In [9]:
print("First five values of train lable : ",target_train_all[:5])

First five values of train lable :  [5 0 4 1 9]


# Data Preprocessing

## re-scaling data between 0 and 1

In [10]:
# it is for features
features_train_all,features_test= features_train_all/255.0 , features_test/255.0

## convert target values into one hot encoding

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

In [11]:
target_train_all= np.eye(NR_CLASSES)[target_train_all]
print(target_train_all)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [12]:
target_test = np.eye(NR_CLASSES)[target_test]
print(target_test)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
print(np.eye(NR_CLASSES)[0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Create validation dataset from training data

In [14]:
features_val = features_train_all[:VALIDATION_SIZE]
target_val = target_train_all[:VALIDATION_SIZE]
print("validation data size: ",features_val.shape)
print("validation target data side: ",target_val.shape)

validation data size:  (10000, 784)
validation target data side:  (10000, 10)


In [15]:
features_train = features_train_all[VALIDATION_SIZE:]
target_train = target_train_all[VALIDATION_SIZE:]
print("Training features data size: ",features_train.shape)
print("Training target data size: ",target_train.shape)

Training features data size:  (50000, 784)
Training target data size:  (50000, 10)


# Setup Tensorflow Graph

In [16]:
#Creating the placeholder to feed the data afterward
FEATURES = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUT], name='FEATURES')
TARGET = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='TARGET')

### NeuralNetwrok Architecture
###### Hyper Parameters


In [1]:
# It will run in 5 throught the data
nr_epoch= 50
# 0.001
learning_rate= 1e-3 
# number of neurons in first hidden layer
n_hidden1=512
# number of neurons in second hidden layer
n_hidden2=64
1e-3 

0.001

In [18]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value= initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b= tf.Variable(initial_value= initial_b, name='B')

        layer_in= tf.matmul(input,w) + b
        
        tf.summary.histogram('weight',w)
        tf.summary.histogram('bias',b)
        
        if name=="out":
            layer_out= tf.nn.softmax(layer_in)
        else:
            layer_out= tf.nn.relu(layer_in)
        return layer_out

In [19]:
# Model 1 Without dropout
# layer_1 = setup_layer(FEATURES, weight_dim=[TOTAL_INPUT,n_hidden1],
#                       bias_dim=[n_hidden1], name='layer_1')
# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1,n_hidden2],
#                       bias_dim=[n_hidden2], name='layer_2')
# output = setup_layer(layer_2, weight_dim=[n_hidden2,NR_CLASSES],
#                       bias_dim=[NR_CLASSES], name='out')
# model_name= f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{epoch}'

In [20]:
layer_1 = setup_layer(FEATURES, weight_dim=[TOTAL_INPUT,n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1,n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2,NR_CLASSES],
                      bias_dim=[NR_CLASSES], name='out')

model_name= f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epoch}'

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [21]:
# Folder for tensorboard
folder_name=f'{model_name} at {strftime("%I:%M")}'
directory= os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else: 
    print("Directory is successfully created!")

Directory is successfully created!


# Loss Optimisation & Matrics

#### Defining loss funciton

In [22]:
with tf.name_scope('losses_calc'):
    loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=TARGET, logits=output))

#### Defining Optimizer

In [23]:
with tf.name_scope('Optimizer'):
    optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate)
    trian_step= optimizer.minimize(loss)

##### Accuracy matrics & summaries

In [24]:
with tf.name_scope('Accuracy_calc'): 
    model_prediction= tf.argmax(output, axis=1, name='prediction')
    correct_pred= tf.equal(model_prediction, tf.argmax(TARGET,axis=1)) # axis 1 is x axis of matrix
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [25]:
with tf.name_scope('Perfomance'):
    tf.summary.scalar('accuracy',accuracy)
    tf.summary.scalar('cost',loss)

In [26]:
with tf.name_scope("show_image"):
    features_image= tf.reshape(FEATURES,[-1,28,28,1])
    tf.summary.image('input image', features_image, max_outputs=4)

INFO:tensorflow:Summary name input image is illegal; using input_image instead.


# Run Session

In [27]:
sess= tf.Session()

#### Setup Filewriter and Merge Summery

In [28]:
merged_summery = tf.summary.merge_all()

train_writer= tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory+ '/validation')


In [29]:
# Initialize all the variables
init= tf.global_variables_initializer()
sess.run(init)

# Batching the data

In [30]:
size_of_batch = 1000

In [31]:
num_examples= target_train.shape[0]
nr_iterations= int(num_examples/size_of_batch)
print("Total iterations: ",nr_iterations)
index_in_epoch=0

Total iterations:  50


In [32]:
def next_batch(batch_size, data, labels):
    
    global num_examples 
    global index_in_epoch 
    
    start=index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch> num_examples:
        start=0
        index_in_epoch= batch_size
    
    
    end= index_in_epoch
    
    return data[start:end], labels[start:end]

# Training Loop

In [33]:
for epoch in range(nr_epoch):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_features, batch_target = next_batch(batch_size=size_of_batch, data=features_train, labels=target_train)
        
        feed_dictionary = {FEATURES:batch_features, TARGET:batch_target}
        
        sess.run(trian_step, feed_dict=feed_dictionary)
        
    
    summary, batch_accuracy = sess.run(fetches=[merged_summery, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(summary, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ======================
    
    summary = sess.run(fetches=merged_summery, feed_dict={FEATURES:features_val, TARGET:target_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.8700000047683716
Epoch 1 	| Training Accuracy = 0.8799999952316284
Epoch 2 	| Training Accuracy = 0.8849999904632568
Epoch 3 	| Training Accuracy = 0.9580000042915344
Epoch 4 	| Training Accuracy = 0.9700000286102295
Epoch 5 	| Training Accuracy = 0.9789999723434448
Epoch 6 	| Training Accuracy = 0.9750000238418579
Epoch 7 	| Training Accuracy = 0.9810000061988831
Epoch 8 	| Training Accuracy = 0.9739999771118164
Epoch 9 	| Training Accuracy = 0.9810000061988831
Epoch 10 	| Training Accuracy = 0.9869999885559082
Epoch 11 	| Training Accuracy = 0.9829999804496765
Epoch 12 	| Training Accuracy = 0.9829999804496765
Epoch 13 	| Training Accuracy = 0.9860000014305115
Epoch 14 	| Training Accuracy = 0.9879999756813049
Epoch 15 	| Training Accuracy = 0.9900000095367432
Epoch 16 	| Training Accuracy = 0.9900000095367432
Epoch 17 	| Training Accuracy = 0.9890000224113464
Epoch 18 	| Training Accuracy = 0.9890000224113464
Epoch 19 	| Training Accuracy = 0.9890000

# Saving the model

In [34]:
outputs= {'accuracy_calc/prediction':model_prediction}
inputs= {'FEATURES': FEATURES}
tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel/saved_model.pb


# Make a Prediction

In [35]:
img= Image.open('MNIST/test_img.png')

In [36]:
bw= img.convert('L')

In [37]:
img_array= np.invert(bw)

In [38]:
img_array.shape

(28, 28)

In [39]:
test_img = img_array.ravel()

In [40]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={FEATURES:[test_img ]})

In [41]:
print(f'Prediciton for the test images is {prediction}')

Prediciton for the test images is [2]


# Testing and Evaluation

In [42]:
test_accuracy= sess.run(fetches=accuracy, feed_dict={FEATURES:features_test, TARGET:target_test})
print(f'Accuracy on test set is : {test_accuracy:0.2%}')

Accuracy on test set is : 97.85%


# Reset for the next run

In [43]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()